<a href="https://colab.research.google.com/github/gauss5930/Natural-Language-Processing/blob/main/BERT/BERT_%EA%B5%AC%ED%98%84_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT 구현 복습

이미 한 번 BERT를 구현했던 적이 있는데, 이번에는 좀 더 구체적인 example을 사용하여 직접 구현해보도록 하겠다. 이 코드는 [여기](https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial)를 참고하여 작성되었다.

BERT를 PyTorch를 이용하여 구현하였고, BERT를 구현하는 과정을 다음과 같이 4개의 섹션으로 나눴다.

1. 전처리
2. 모델링
3. Loss & Optimization
4. 훈련


### 전처리

전처리 과정에서는 신경망이 데이터를 처리할 수 있도록 다음과 같이 data를 구축한다. 일단 raw text부터 시작해보도록 하자.

In [ ]:
# raw text

text = (
       'Hello, how are you? I am Romeo.n'
       'Hello, Romeo My name is Juliet. Nice to meet you.n'
       'Nice meet you too. How are you today?n'
       'Great. My baseball team won the competition.n'
       'Oh Congratulations, Julietn'
       'Thanks you Romeo'
   )

그 다음에 데이터를 다음과 같이 정리해야 한다.

- 문장을 소문자로 변환
- vocabulary를 만듦. **Vocabulary**는 문서 내의 독특한 단어의 list임.

In [ ]:
# '.', ',', '?', '!' filtering
sentences = re.sub("[.,!?-]", '', text.lower()).split('n')

word_list = list(set(" ".join(sentences).split()))

다음으로, BERT의 학습 도중에 사용되는 special token을 잘 기억해야 한다. 다음은 이 다양한 토큰들에 대한 설명이다.

- [CLS]: 첫 번째 토큰은 항상 classification
- [SEP]: 두 개의 문장을 분리
- [END]: 문장을 끝내기
- [PAD]: 문장을 똑같은 길이로 줄이기
- [MASK]: 기존의 단어를 mask로 대체

이러한 토큰들은 word dictionary에 들어가 있어야 하는데, 여기서 vocabulary에 들어가 있는는 각각의 토큰과 단어는 index number가 할당된다.

In [ ]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_list):
  word_dict[w] = i + 4
  number_dict = {i: w for i, w in enumerate(word_dict)}
  vocab_size = len(word_dict)

이 과정이 완료되면, input sequence를 3개의 유형의 embedding으로 포맷하는 함수를 생성해야 한다.

- **token embedding**
- **segment embedding**
- **position embedding**

이제 각각에 대해 알아보도록 하자.

**token embedding이 무엇일까?**

예를 들어, 문장 "The cat is walking. The dog is barking."이 주어졌을 때, 함수는 다음의 방식대로 sequence를 생성해야 한다.

"[CLS] the cat is walking [SEP] the dog is barking"

그 후에, 모든 것들은 word dictionary의 index로 바꿔야 한다. 따라서 이전의 문장은 다음과 같은 형태를 가지게 된다.

"[1, 5, 7, 9, 10, 2, 5, 6, 9, 11]"

여기서 1과 2는 각각 [CLS]와 [SEP]를 의미한다.

**segment embedding이 무엇일까?**

segment embedding은 두 개의 문장을 분리하는 역할을 한다. 보통 0과 1로 정의된다.

**position embedding이 무엇일까?**

position embedding은 sequence에서 각 embedding에게 position을 준다.



이제 다음 단계는 **masking**을 생성하는 것이다.

논문에 의하면, BERT는 sequence의 15% word를 [MASK] 토큰으로 대체하고, padding을 추가하였다. Padding은 모든 문장의 길이를 똑같은 길이로 만들어준다. 예를 들어, 다음과 같은 문장을 받았다고 하였을 때,

"The cat is walking. The shog is barking at the tree"

이 문장에 padding을 적용하면 다음과 같이 바뀐다.

"[CLS] The cat is walking [PAD] [PAD] [PAD]. [CLS] The dog is barking at the tree."

첫 번째 문장의 길이가 두 번째 문장의 길이와 같아진다.

In [ ]:
def make_batch():
  batch = []
  positive = negative = 0
  while positive != batch_size / 2 or negative != batch_size / 2:
    tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))

    tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

    input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP']]
    segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

    # LM masking
    n_pred = min(max_pred, max(1, int(round(len(input_ids) * 0.15))))   # 한 문장의 15% 정도의 토큰
    cand_maked_pos = [1 for i, token in enumerate(input_ids) if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
    shuffle(cand_maked_pos)
    masked_tokens, masked_pos = [], []
    for pos in cand_makes_pos[:n_pred]:
      masked_pos.append(pos)
      masked_tokens.append(input_ids[pos])
      if random() < 0.8:   # 80%는 masking
        input_ids[pos] = word_dict['[MASK]']
      elif random() < 0.5: # 10%는 vocabulary에서 random indexing
        index = randint(0, vocab_size - 1)
        input_ids[pos] = word_dict[number_dict[index]]
    
    # Zero padding
    n_pad = maxlen - len(input_ids)
    input_ids.extend([0] * n_pad)
    segment_ids.extend([0] * n_pad)

    # Zero padding (100% - 15%) tokens
    if max_pred > n_pred:
      n_pad = max_pred - n_pred
      masked_tokens.extend([0] * n_pad)
      masked_pos.extend([0] * n_pad)

    if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
      batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])   # IsNext
    elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
      batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])  # NotNext
      negative += 1

    return batch

next-word prediction을 다루기 때문에, 문장이 이어진 문장인지 아닌지를 예측하는 label을 생성해야 한다. 이것이 바로 IsNext와 NotNext이다. 그래서 다음 문장 앞에 오는 모든 문장에 True를 할당하고 이를 위해 조건문을 사용하였다.

예를 들어, 두 개의 문장이 하나의 document에 있으면, 이 둘은 서로를 문맥적으로 따른다. 따라서서 첫 번째 문장이 A이면 다음 문장은 A+1이어야 한다. 직관적으로 첫 번째 문장의 위치 즉, tokens_a_index + 1 == tokens_b_index, 즉 동일한 context의 두 번째 문장인 경우 이 입력에 대한 label을을 True로 설정할 수 있도록 코드를 작성해야 한다.

만약 위 조건이 tokens_a_index + 1 != tokens_b_index라면 input에 대한 label을 False로 지정해야 한다.

### 모델링

BERT는 매우 정교한 모델이라서 느리게 감지되면 논리를 잃게 된다. 그래서 BERT는 component와 함수에 의해 component를 설명하는 것이 가능하다.

BERT는 다음의 component들을 가진다.

1. Embedding layer
2. Attention Mask
3. Encoder layer
  - Multi-head attention
    - Scaled dot product attention
  - Position-wise feed-forward network
4. BERT(모든 component를 합침)

#### Embedding Layer

embedding은 BERT의 첫 번째 레이어로 input을 받아서 lookup table을 생성한다. embedding layer의 파라미터는 학습 가능하고, 이는 학습 스포레스가 끝날 때, embedding은 비슷한 단어들끼리 모여있을 거라는 것이다.

embedding layer는 단어 간의 서로 다른 관계를 보존한다. 여기에는 semantic, syntactic, linear, 그리고 BERT가 양방향성이기 때문에, contextual relationship을 잘 보존한다.

BERT의 경우에, 다음 3개의 embedding을 생성한다.

- Token
- Segments
- Position

아까 전에 position embedding을 생성하는 함수를 정의해두지는 않았지만, token과 segment를 생성하는 함수는 이미 정의해두었다. 그래서 이제 input을 받아서 sequence에서 각 단어에 대한 position을 생성할 수 있다. 그리고 이는 다음과 같다.

In [ ]:
print(torch.arange(30, dtype = torch.long).expand_as(input_ids))

forward function에서, 모든 embedding을 합하고 정규화하였다.

In [ ]:
class Embedding(nn.Module):
  def __init__(self):
    super(EMbedding, self).__init__()
    self.tok_embed = nn.Embedding(vocab_size, d_model)   # token embedding
    self.pos_embed = nn.Embedding(maxlen, d_model)       # position embedding
    self.seg_embed = nn.Embedding(n_segments, d_model)   # segment embedding
    self.norm = nn.LayerNorm(d_model)

  def forward(self, x, seg):
    seq_len = x.size(1)
    pos = torch.arange(seq_len, dtype = torch.long)
    pos = pos.unsqueeze(0).expand_as(x)                  # (seq_len,) -> (batch_size, seq_len)
    embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)

    return self.norm(embedding)

#### attention mask 생성

BERT는 attention mask 또한 필요로 한다. 그리고 이것은 적절한 형식이 되어야 한다. 다음의 코드가 attention mask를 생성하는 코드이다. 아래 코드에서 [PAD]는 1로 변환되고, 다른 것들은 0으로 변환된다.

In [ ]:
def get_attn_pad_mask(seq_q, seq_k):
  batch_size, len_q = seq_q.size()
  batch_size, len_k = seq_k.size()
  # eq(0)은 PAD token이다.
  pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)          # batch_size x 1 x len_k(=len_q), 하나가 마스킹된다.
  return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

#### Encoder

Encdoer는 다음의 두 개의 주된 component를 가지고 있다.

- Multi-head Attention
- Position-wise feed-forward network

encoder의 작업은 representation과 pattern을 input과 attention mask로부터 찾는 것이다.

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self):
    super(EncoderLayer, self).__init__()
    self.enc_self_attn = MultiHeadAttention()
    self.pos_ffn = PoswiseFeedForwardNet()

  def forward(self, en_inputs, enc_self_attn_mask):
    enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)   # enc_inputs는 Q, K, V와 같음
    enc_outputs = self.pos_ffn(enc_outputs)   # enc_outputs: [batch_size x len_q x d_model]
    return enc_outputs, attn

#### Multi-head attention

이것이 encoder의 첫 번째 주된 component이다.

attention model은 3개의 입력값 **Query, Key, Value**를 받는다.

Multi-head attention은 4개의 입력값 **Query, Key, Value, Attention mask**를 받는다. embedding은 Query, Key, Value에 입력으로 주어지고, attention mask는 attention mask 인자에 입력으로 주어진다.

이러한 3개의 입력과 attention mask에 대해 dot-product 연산을 수행한다. 이 dot-product 연산은 **context vector**와 **attention**을 산출한다. context vector는 선형 레이어를 지나서 최종적으로 output을 출력한다.

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self):
    super(MultiHeadAttention, self).__init__()
    self.W_Q = nn.Linear(d_model, d_k * n_heads)
    self.W_K = nn.Linear(d_model, d_k * n_heads)
    self.W_V = nn.Linear(d_model, d_v * n_heads)

  def forward(self, Q, K, V, attn_mask):
    # q: [batch_size x len_q x d_model]
    # k: [batch_size x len_k x d_model]
    # v: [batch_size x len_k x d_model]
    residual, batch_size = Q, Q.size(0)
    # (B, S, D) -proj- -> (B, S, D) -split- -> (B, S, H, W) -trans- -> (B, H, S, W)
    q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)   # q_s: [batch_size x n_heads x len_q x d_k]
    k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)   # k_s: [batch_size x n_heads x len_k x d_k]
    v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)   # v_s: [batch_size x n_heads x len_k x d_v]

    attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)            # attn_mask: [batch_size x n_heads x len_q x len_k]

    # context: [batch_size x n_heads x len_q x d_v]
    # attn: [batch_size x n_heads x len_q x len_k]
    context, attn = ScaleDotProductAttention()(q_s, k_s, v_s, attn_mask)
    context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)   # context: [batch_size x len_q x n_heads * d_v]
    output = nn.Linear(n_heads * d_v, d_model)(context)

    return nn.LayerNorm(d_model)(output + residual), attn   # output: [batch_size x len_q x d_model]

이제 이 Scaled Dot-Product attention에 대해 알아보도록 하자.

- scaled dot-product attention 클래스는 4개의 인자 Query, Key, Value, Attention mask를 받는다. 본질적으로, 앞에 3개의 인자들은 word embedding과 함께 주어지고, attention mask 인자는 attention mask embedding과 함께 주어진다.
- 그리고 scaled dot-product attention은 **query**와 **key**간에 행렬곱을 해서 점수를 얻는다.

우리 코드에서는 scores.masked_fill_(attn_mask, -1e9)를 사용한다. 이 속성은 attention mask가 **True**인 -1e9로 score 요소를 채우고 나머지 요소는 attention score를 얻은 다음 0과 1 사이의 score를 제공하는 softmax 함수를 통해 전달된다.마지막으로, attention 과 value 간에 행렬곱을 수행함으로써 context vector을 얻었다.

In [ ]:
class ScaledDotProductAttention(nn.Module):
  def __init__(self):
    super(ScaledDotProductAttention, self).__init__()

  def forward(self, Q, K, V, attn_mask):
    scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)   # scores: [batch_size x n_heads x len_q x len_k]
    scores.masked_fill_(attn_mask, -1e9)   # mask가 하나인 self tensor의 요소를 value로 채운다.
    attn = nn.Softmax(dim = -1)(scores)
    context = torch.matmul(attn, V)
    return score, context, attn
    

#### Position-Wise Feed Forward Network

multi-head attention의 출력값은 feed-forward network로 가고 이는 encoder part를 결론 짓는다.

#### 모든 component를 합치기

encoder는 다음의 2개의 출력값을 내놓는다.

- feed-forward layer의 출력값
- Attention mask

여기서 중요한 것은 BERT는 decoder를 사용하지 않는다는 것이다. 대시넹, output과 attention mask를 사용해서 원하는 결과를 얻는다.

transformer의 decoder 부분은 아래 코드처럼 분류하는데 사용되는 얕은 네트워크로 대체된다. BERT 또한 **classifier**와 **masked** 2개의 출력값을 내놓는다.

In [ ]:
class BERT(nn.Module):
  def __init__(self):
    super(BERT, self).__init__()
    self.embedding = Embedding()
    self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
    self.fc = nn.Linear(d_model, d_model)
    self.activ1 = nn.Train()
    self.linear = nn.Linear(d_model, d_model)
    self.activ2 = gelu
    self.norm = nn.LayerNorm(d_model)
    self.classifier = nn.Linear(d_model, 2)
    # decoder는 embedding layer와 공유됌
    embed_weight = self.embedding.tok_embed.weight
    n_vocab, n_dim = embed_weight.size()
    self.decoder = nn.Linear(n_dim, n_vocab, bias = False)
    self.decoder.weight = embed_weight
    self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

  def forward(self, input_ids, segment_ids, masked_pos):
    output = self.embedding(input_ids, segment_ids)
    enc_self_attn_mask = deg_attn_pad_mask(input_ids, input_ids)
    for layer in self.layers:
      output, enc_self_attn = layer(output, enc_self_attn_mask)
      # output: [batch_size, len, d_model]
      # attn: [batch_size, n_heads, d_model, d_model]
      # 이는 첫 번째 토큰 (CLS)에 의해 결정됌
    h_pooled = self.activ1(self.fc(output[:, 0]))   # [batch_size, d_model]
    logits_clsf = self.classification(h_pooled)     # [batch_size, 2]

    masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))   # [batch_size, max_pred, d_model]

    # transformer의 최종 출력으로부터 masked position을 얻음
    h_masked = torch.gather(output, 1, masked_pos)   # masking position: [batch_size, max_pred, d_model]
    h_masked = self.norm(self.activ2(self.linear(h_masked)))
    logits_lm = self.decoder(h_masked) + self.decoder_bias   # [batch_size, max_pred, n_vocab]

    return logits_lm, logits_clsf

몇 가지 기억해두어야 할 사항이 있다.

1. encoder의 수를 지정할 수 있다. 논문에서는 base model의 경우 12개였다.
2. BERT에는 2개의 활성화 함수가 있는데, Tanh와 GELU이다.

In [ ]:
def gelu(x):
  return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

### Loss & Optimization

논문에서는 모든 vocabulary에 대해 확률 분포를 계산하였지만, softmax 근사치를 이용해서 계산이 가능하다. 하지만, 확률 분포를 구하는 깔끔한 방법은 **cross-entropy**를 사용하는 것이다. cross-entropy loss는 *softmax*와 *negative log-likelihood*의 조합이다.

그래서 모델을 구축하는 동안 softmax를 포함할 필요 없이 softmax 정규화 없이 feed-forward network에서 깔끔한 출력을 얻을 수 있다. 

optimization으로 넘어가서 BERT에서는 Adam optimizer를 사용하였다.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

### 훈련

마지막으로 모델 훈련을 해보도록 하자.

In [ ]:
model = BERT()
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

  for epoch in range(100):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens)   # masked LM을 위해
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext)   # sentence classification을 위해
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
      print('Epoch:', '%04d' % (epoch + 1), 'cost = ', '{:.6f}'.format(loss))
      loss.backward()
      optimizer.step()

    # mask token 예측하기
    input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
    print(text)
    print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    logits_lm = logits_lm.data.mix(2)[1][0].data.numpy()
    print('masked tokens list: ', [pos.item() for pos in masked_tokens[0] if pos.item() != 0])
    print('predict masked tokens list: ', [pos for pos in logits_lm if pos != 0])

    logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
    print('isNext: ', True if isNext else False)
    print('predict isNext: ', True is logits_clsf else False)

In [ ]:
Output:

Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thanks you Romeo
['[CLS]', 'nice', 'meet', 'you', 'too', 'how', 'are', 'you', 'today', '[SEP]', '[MASK]', 'congratulations', '[MASK]', '[SEP]']
masked tokens list :  [27, 22]
predict masked tokens list :  []
isNext :  False
predict isNext :  True

이렇게 해서 BERT를 모두 구현하였다. 좀 더 큰 corpus에 대해서도 똑같은 BERT 모델을 사용할 수 있다.

1. Pre-training: corpus를 사용하지만 앞서 언급한 input representation의 정확한 형식을 사용
2. FIne-tuning: 지도학습 데이터를 사용해야 한다.
3. 다양한 task 또는 topic modeling을 위한 feature extractor가 있어야 함